In [1]:
import tkinter as tk
from tkinter import PhotoImage
import webbrowser
import subprocess
import boto3
import datetime
import tkinter as tk
from tkinter import PhotoImage
import webbrowser
import pywhatkit
import subprocess
from tkinter import PhotoImage, Scrollbar

# Function to open Google Chrome
def open_tab():
    subprocess.Popen(['chrome.exe'])

# Function to open the Calculator
def open_calculator():
    subprocess.Popen(['calc.exe'])

# Function to open Command Prompt (cmd)
def open_cmd():
    subprocess.getoutput("start cmd")

def open_chrome():
    subprocess.getoutput("start chrome")

# Function to send a WhatsApp message
def send_whatsapp_message():
    # Get the input from the text box and split it into a list using commas as separators
    input_text = input_entry.get()
    data = input_text.split(',')

    if len(data) != 4:
        print("Please enter data in the correct format: Number, Message, Hour, Minute")
        return

    receiver_number = str(data[0].strip())  # Convert to string and remove leading/trailing spaces
    message = data[1].strip() if data[1].strip() else "Tarun Gupta here"
    # Remove leading/trailing spaces
    hour = int(data[2].strip())  # Remove leading/trailing spaces and convert to int
    minute = int(data[3].strip())  # Remove leading/trailing spaces and convert to int

    pywhatkit.sendwhatmsg(receiver_number, message, hour, minute)
    print("Message sent to", receiver_number)

# Function to perform a Google search
def google_search():
    query = search_entry.get()  # Get the text from the entry field
    webbrowser.open_new("https://www.google.com/search?q=" + query)
    
# Initialize the Polly and S3 clients
polly_client = boto3.client('polly')
#s3_client = boto3.client('s3', region_name='ap-south-1')  # Specify the correct region

myec2 = boto3.client("ec2")

import boto3

def launch_ec2_with_input():
    user_input = ec2_input.get()  # Get the text from the EC2 input field
    input_data = user_input.split(',')

    if len(input_data) == 2 and input_data[0].strip() in ['1', '2', '3']:
        choice = int(input_data[0].strip())
       
        if choice == 1:
            instance_name = input_data[1].strip()
            response = myec2.run_instances(
                ImageId='ami-0da59f1af71ea4ad2',
                InstanceType='t2.micro',
                MaxCount=1,
                MinCount=1
            )
            instance_id = response['Instances'][0]['InstanceId']
            myec2.create_tags(Resources=[instance_id], Tags=[{'Key': 'Name', 'Value': instance_name}])
            update_result_text(f"EC2 instance launched with instance ID: {instance_id}\nInstance name: {instance_name}")

        elif choice == 2:
            num_instances = int(input_data[1].strip())
            
            response = myec2.run_instances(
                ImageId='ami-0da59f1af71ea4ad2',
                InstanceType='t2.micro',
                MaxCount=num_instances,
                MinCount=num_instances
            )
            instance_ids = [instance['InstanceId'] for instance in response['Instances']]
            update_result_text(f"Launched {num_instances} EC2 instances with instance IDs: {', '.join(instance_ids)}")

        elif choice == 3:
            response = myec2.describe_instances()
            instance_info = []
            for reservation in response['Reservations']:
                for instance in reservation['Instances']:
                    instance_info.append({
                        'Instance ID': instance['InstanceId'],
                        'Instance Type': instance['InstanceType'],
                        'State': instance['State']['Name']
                    })
            result_text = "Instance information:\n"
            for info in instance_info:
                result_text += f"{info}\n"
            update_result_text(result_text)

        else:
            update_result_text("Invalid choice. Please enter '1', '2', or '3'.")
    else:
        update_result_text("Please enter data in the correct format: Choice, Additional Data")


# Example usage:
# ec2 =   1 , name    >   1, MyInstance
# ec2 =   2 , number  >   2, 3
# ec2 =   3 , blank   >   3

# You can call this function with user input like this:
# launch_ec2()

# Function to create an S3 bucket

region_name = 'ap-south-1'
s3_client = boto3.client('s3', region_name=region_name)

def create_s3_bucket():
    print("Press y to create an S3 bucket and name it.")
    print("Press n to not create an S3 bucket.")
    print("Press m to create multiple S3 buckets with serial numbers.")
    print("Press i for information about S3 buckets")

    choice = input("Enter your choice (y/n/m/i): ")

    if choice.lower() == "y":
        bucket_name = input("Enter your unique bucket name: ")
        try:
            # Specify the region constraint when creating the bucket
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region_name})
        except s3_client.exceptions.BucketAlreadyExists:
            print("Bucket name already exists. Please choose a different name.")
        except s3_client.exceptions.BucketAlreadyOwnedByYou:
            print("You already own a bucket with this name.")
        except Exception as e:
            print("An error occurred:", e)

    elif choice.lower() == "n":
        print("No S3 bucket will be created.")

    elif choice.lower() == "m":
        while True:
            num_buckets_input = input("Enter the number of buckets you want to create: ")
            if num_buckets_input.isdigit():
                num_buckets = int(num_buckets_input)
                break
            else:
                print("Invalid input. Please enter a valid number.")

        base_name = input("Enter the base name for the buckets: ")

        for i in range(1, num_buckets + 1):
            bucket_name = f"{base_name}-{i}"
            try:
                #s3_client.create_bucket(Bucket=bucket_name)
                # Specify the region constraint when creating the bucket
                s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region_name})
                print(f"S3 bucket {i} created:", bucket_name)
            except s3_client.exceptions.BucketAlreadyExists:
                print(f"Bucket {bucket_name} already exists. Skipping...")
            except s3_client.exceptions.BucketAlreadyOwnedByYou:
                print(f"You already own a bucket with the name {bucket_name}. Skipping...")
            except Exception as e:
                print(f"An error occurred while creating bucket {bucket_name}:", e)

    elif choice.lower() == "i":
        response = s3_client.list_buckets()
        bucket_info = []
        for bucket in response['Buckets']:
            bucket_info.append({
                'Bucket Name': bucket['Name'],
                'Creation Date': bucket['CreationDate']
            })
        print("S3 bucket information:")
        for info in bucket_info:
            print(info)

    else:
        print("Invalid choice. Please enter 'y', 'n', 'm', or 'i'.")

# Function to synthesize speech using Polly
def synthesize_speech():
    text = input("Paste / write your text here: ")
    print("Choose voice:")
    print("1. Male (Kevin)")
    print("2. Female (Joanna)")
    choose = int(input("Enter the option: "))

    voice_id = None
    if choose == 1:
        voice_id = 'Matthew'
    elif choose == 2:
        voice_id = 'Joanna'
    else:
        print("Invalid option")
        return

    output_format = 'mp3'

    # Request the speech synthesis
    response = polly_client.synthesize_speech(
        Text=text,
        VoiceId=voice_id,
        OutputFormat=output_format
    )

    # Get the audio stream
    audio_stream = response['AudioStream'].read()

    # Save the audio to a file with a unique filename
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    output_filename = f'voice-aws-{timestamp}.mp3'
    with open(output_filename, 'wb') as file:
        file.write(audio_stream)

    print(f'Speech saved to {output_filename}')

    # User choices for playing, saving locally, or saving to S3
    print("\nChoose an option:")
    print("1. Save in S3 Bucket")
    user_choice = int(input("Enter the option: "))

    if user_choice == 1:
        # Save the audio to an S3 bucket
        bucket_name = 'tarun-vioce-ai'  # Replace with your S3 bucket name
        s3_client.upload_file(output_filename, bucket_name, output_filename)
        s3_url = f'https://{bucket_name}.s3.ap-south-1.amazonaws.com/{output_filename}'
        print(f'Speech saved to S3 bucket: {s3_url}')
    else:
        print("Invalid option")

# Create the main window
root = tk.Tk()
root.title("Open Applications")
root.geometry("1000x500")  # Set window size

# Load the background image
background_image = PhotoImage(file=r"C:\Users\TaRuN\Desktop\linux world summer intenship\task 2\4.png")  # Use raw string (r)
# Create a label widget to display the background image
background_label = tk.Label(root, image=background_image)
background_label.place(relwidth=1, relheight=1)  # Make the label cover the entire window

# Define button and input field colors
button_bg_color = "#A2FF7E"  # Light Green background color
button_text_color = "black"   # Black text color
input_bg_color = "white"      # White background color for the input field


# Create buttons with specified colors
button_width = 20
button_height = 1

chrometab_button = tk.Button(root, text="Open Chrome tab", command=open_chrome, width=button_width, height=button_height, bg=button_bg_color, fg=button_text_color)
calculator_button = tk.Button(root, text="Open Calculator", command=open_calculator, width=button_width, height=button_height, bg=button_bg_color, fg=button_text_color)
cmd_button = tk.Button(root, text="Open cmd", command=open_cmd, width=button_width, height=button_height, bg=button_bg_color, fg=button_text_color)
chrome_button = tk.Button(root, text="open chrome", command=open_chrome, width=button_width, height=button_height, bg=button_bg_color, fg=button_text_color)

# Create a text box for user input
input_entry = tk.Entry(root, bg=input_bg_color, width=50)

# Create a "Send WhatsApp Message" button
whatsapp_button = tk.Button(root, text="Send WhatsApp Message", command=send_whatsapp_message, bg=button_bg_color, fg=button_text_color)


# Create an Entry widget for Google search input
search_entry = tk.Entry(root, bg=input_bg_color, width=30)

# Create a "Google It" button for searching
google_button = tk.Button(root, text="Google It", command=google_search, bg=button_bg_color, fg=button_text_color)


# Use the grid manager to position the widgets
chrometab_button.grid(row=0, column=0, padx=10, pady=10)
calculator_button.grid(row=0, column=1, padx=10, pady=10)
cmd_button.grid(row=0, column=2, padx=10, pady=10)
chrome_button.grid(row=0, column=3, padx=10, pady=10)
input_entry.grid(row=1, column=0, columnspan=2, padx=10, pady=10)
whatsapp_button.grid(row=1, column=1, columnspan=2, padx=10, pady=10)
search_entry.grid(row=2, column=0, padx=10, pady=10)
google_button.grid(row=2, column=1, padx=10, pady=10)

# Create buttons for AWS services

# Create a "Launch EC2" button
ec2_button = tk.Button(root, text="Launch EC2", command=launch_ec2_with_input, width=button_width, height=button_height, bg=button_bg_color, fg=button_text_color)
s3_button = tk.Button(root, text="Create S3 Bucket", command=create_s3_bucket, width=button_width, height=button_height, bg=button_bg_color, fg=button_text_color)
polly_button = tk.Button(root, text="Synthesize Speech (Polly)", command=synthesize_speech, width=button_width, height=button_height, bg=button_bg_color, fg=button_text_color)

# Create input fields for AWS services
ec2_input = tk.Entry(root, bg=input_bg_color, width=50)
s3_input = tk.Entry(root, bg=input_bg_color, width=50)
polly_input = tk.Entry(root, bg=input_bg_color, width=50)

# Create a label to display the results
#result_label = tk.Label(root, text="", bg=button_bg_color, fg=button_text_color)
# Create a label to display the results
#result_label = tk.Label(root, text="", bg=button_bg_color, fg=button_text_color)
#result_label.grid(row=9, column=0, columnspan=3, padx=10, pady=10)  # Adjust the row and column as needed



# Create a text widget for displaying results with vertical and horizontal scrollbars
result_text = tk.Text(root, bg=button_bg_color, fg=button_text_color, font=("Helvetica", 10), wrap=tk.WORD)
result_text.grid(row=9, column=0, columnspan=3, padx=10, pady=10, sticky="nsew")

# Create a vertical scrollbar for the text widget
scrollbar_y = Scrollbar(root, command=result_text.yview)
scrollbar_y.grid(row=9, column=3, sticky="ns")

# Create a horizontal scrollbar for the text widget
scrollbar_x = Scrollbar(root, command=result_text.xview, orient=tk.HORIZONTAL)
scrollbar_x.grid(row=10, column=0, columnspan=3, sticky="ew")

# Configure the text widget to use both vertical and horizontal scrollbars
result_text.config(yscrollcommand=scrollbar_y.set, xscrollcommand=scrollbar_x.set)

# Function to update the text in the result_text widget
def update_result_text(text):
    result_text.config(state=tk.NORMAL)  # Set the text widget to editable
    result_text.delete(1.0, tk.END)  # Clear the current text
    result_text.insert(tk.END, text)  # Insert the new text
    result_text.config(state=tk.DISABLED)  # Set the text widget to read-only


# Now, you can update the result_text widget using the update_result_text function
# Example usage:
# update_result_text("This is some text that can be scrolled up and down.\n" * 10)  # Add your actual text here



# Create a label to display the results with a larger font size
result_label = tk.Label(root, text="", bg=button_bg_color, fg=button_text_color, font=("Helvetica", 16))  # You can adjust the font size (16) as needed
result_label.grid(row=9, column=0, columnspan=3, padx=10, pady=10)  # Adjust the row and column as needed




# Create labels for input fields
ec2_label = tk.Label(root, text="EC2 Input:", bg=button_bg_color, fg=button_text_color)
s3_label = tk.Label(root, text="S3 Input:", bg=button_bg_color, fg=button_text_color)
polly_label = tk.Label(root, text="Polly Input:", bg=button_bg_color, fg=button_text_color)


# Use the grid manager to position the AWS service widgets
ec2_label.grid(row=6, column=0, padx=10, pady=10)
ec2_input.grid(row=6, column=1, padx=10, pady=10)
ec2_button.grid(row=6, column=2, padx=10, pady=10)

s3_label.grid(row=7, column=0, padx=10, pady=10)
s3_input.grid(row=7, column=1, padx=10, pady=10)
s3_button.grid(row=7, column=2, padx=10, pady=10)

polly_label.grid(row=8, column=0, padx=10, pady=10)
polly_input.grid(row=8, column=1, padx=10, pady=10)
polly_button.grid(row=8, column=2, padx=10, pady=10)

# Start the Tkinter main loop
root.mainloop()


'''
note 
to use this you need to configure AWS CLI in you system 
and need to change AWS bucket region , name according to you setup
the first 4 option can vary from person to person

the ideal input after you have completed the setup
would look like this

                          text box 
option 1 =   ec2 : 1 , instance-name
option 2 =   ec2 : 2 , number-of-instance
option 3 =   ec2 : 3 , blank

'''


'\nnote \nto use this you need to configure AWS CLI in you system \nand need to change AWS bucket region , name according to you setup\nthe first 4 option can vary from person to person\n\nthe ideal input after you have completed the setup\nwould look like this\n\n                          text box \noption 1 =   ec2 : 1 , instance-name\noption 2 =   ec2 : 2 , number-of-instance\noption 3 =   ec2 : 3 , blank\n\n'